In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy

In [2]:
# Creat 1500 random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=1500)
lngs = np.random.uniform(-180, 180, size=1500)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [3]:
# Create a list of cities' name
cities = []

# Indentify the nearest cities for the coordinates
for i in coordinates:
    city = citipy.nearest_city(i[0], i[1]).city_name
    
    # Add city to the list
    if city not in cities:
        cities.append(city)
        
# Show how many cities we detect
len(cities)

611